In [1]:
!pip install -q keras-nlp --upgrade

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import keras_nlp
import keras
import tensorflow as tf
import os
import gc

keras.mixed_precision.set_global_policy("mixed_float16")

In [3]:
DATA_DIR = '/kaggle/input/contradictory-my-dear-watson/'
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [4]:
df_train = pd.read_csv(DATA_DIR +"train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [6]:
def split_labels(x, y):
    return (x[0], x[1]), y

- Use this block if the preprocessor used for the model is changed from bert base multi

In [7]:
batch_size = 28
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

Size of the train dataset: 12120


In [8]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=265)

bert_train_set = (train_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))

classifier = keras_nlp.models.BertClassifier.from_preset("bert_base_multi", preprocessor=None, num_classes=3)
classifier.compile(optimizer=keras.optimizers.Adam(4e-05), metrics=['accuracy'])
history = classifier.fit(bert_train_set, epochs=5)

Mounting files to /kaggle/input/bert/keras/bert_base_multi/2...
Epoch 1/5


I0000 00:00:1730584791.101978     105 service.cc:145] XLA service 0x79f57c01dec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730584791.102044     105 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730584791.102049     105 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1730584838.361118     148 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32892', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1730584890.023596     105 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


432/432 ━━━━━━━━━━━━━━━━━━━━ 327s 440ms/step - accuracy: 0.5149 - loss: 0.9756
Epoch 2/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 193s 446ms/step - accuracy: 0.6814 - loss: 0.7378
Epoch 3/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 193s 447ms/step - accuracy: 0.7853 - loss: 0.5344
Epoch 4/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 193s 448ms/step - accuracy: 0.8498 - loss: 0.4004
Epoch 5/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 193s 447ms/step - accuracy: 0.8892 - loss: 0.2898


### Precitions on test data for submission

In [11]:
df_test = pd.read_csv(DATA_DIR +"test.csv")
print(df_test.head())
df_test["label"] = [0] * len(df_test)

           id                                            premise  \
0  c6d58c3f69  بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...   
1  cefcc82292                             هذا هو ما تم نصحنا به.   
2  e98005252c  et cela est en grande partie dû au fait que le...   
3  58518c10ba                   与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp   
4  c32b0d16df                              Она все еще была там.   

                                          hypothesis lang_abv language  
0  کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...       ur     Urdu  
1  عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...       ar   Arabic  
2                             Les mères se droguent.       fr   French  
3                            IMA与其他组织合作，因为它们都依靠共享资金。       zh  Chinese  
4     Мы думали, что она ушла, однако, она осталась.       ru  Russian  


In [14]:
def split_labels(x, y):
    return (x[0], x[1]), y

testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=265)

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(1, drop_remainder=False).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))




5195


In [15]:
predictions = classifier.predict(bert_test_set)

I0000 00:00:1730585992.640006     262 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_99', 88 bytes spill stores, 88 bytes spill loads



5195/5195 ━━━━━━━━━━━━━━━━━━━━ 49s 8ms/step


In [16]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

[2 1 0 ... 1 0 2]


In [18]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,2
5192,1f0ea92118,1
5193,0407b48afb,0
